In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import random_split
from torch.utils.data import TensorDataset, DataLoader
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import optuna


# Load data
path = Path('/Users/katesautel/Documents/Seminar_2024/rel_z.csv')
df = pd.read_csv(path)


# Split data into features (F) and target (T)
F = df.iloc[:-1, 0:5]
T = df.iloc[:-1, 5]


# Scale features to be between 0 and 1
scaler = MinMaxScaler()
F_scaled = scaler.fit_transform(F)


# Convert data from arrays into tensors
F_tensor = torch.FloatTensor(F_scaled)
T_tensor = torch.FloatTensor(T.values)


# Combine F and T into single dataset
data = TensorDataset(F_tensor, T_tensor)


# Determine training vs. test set split (50%/50%)
train_size = len(data) // 2
test_size = len(data) - train_size


# Randomly split data into train and test
train_data, test_data = random_split(dataset = data, lengths = [train_size, test_size])


# Define neural network
class NeuralNetwork(nn.Module):
    def __init__(self, num_input_features, num_hidden_neurons, num_hidden_layers):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(num_input_features, num_hidden_neurons)
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(num_hidden_neurons, num_hidden_neurons) for n in range(num_hidden_layers)])
        self.output_layer = nn.Linear(num_hidden_neurons, 1)

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.output_layer(x)
        return x


# Create objective function for Optuna
def objective(trial):
    # Define hyperparameters to optimize
    num_hidden_neurons = trial.suggest_int('num_hidden_neurons', 16, 256)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    learning_rate = trial.suggest_float('learning_rate', 0.00001, 0.1, log = True)
    num_epochs = trial.suggest_int('num_epochs', 0, 500)
    momentum = trial.suggest_float('momentum', 0, 1)
    
    # Initialize model
    model = NeuralNetwork(num_input_features = 5,
                          num_hidden_neurons = num_hidden_neurons, 
                          num_hidden_layers = num_hidden_layers)
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        for inputs, targets in train_dl:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            loss.backward()
            optimizer.step()

    # Evaluate model performance
    model.eval()
    with torch.no_grad():
        y_true = [] 
        y_pred = []
        for inputs, targets in test_dl:
            outputs = model(inputs)
            y_true.extend(targets.numpy())
            y_pred.extend(outputs.numpy().squeeze())

    mse = mean_squared_error(y_true, y_pred)
    return mse


# Optuna study
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 100)


# Get best hyperparameters
best_params = study.best_params
num_hidden_neurons = best_params['num_hidden_neurons']
num_hidden_layers = best_params['num_hidden_layers']
learning_rate = best_params['learning_rate']


# Train model with best hyperparameters
model = NeuralNetwork(num_input_features = 5, 
                      num_hidden_neurons = num_hidden_neurons, 
                      num_hidden_layers = num_hidden_layers)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for inputs, targets in train_dl:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

!pip install optuna